# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [4]:
# inserted cell for data exploration
len(text)
text[:500]

"Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.\nMoe_Szyslak: What's the matter Homer? You're not your normal effervescent self.\nHomer_Simp"

In [5]:
view_sentence_range = (500, 550)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 500 to 550:
Barney_Gumble: (CALLING AFTER HIM) Hey, Joey Joe Joe!
Homer_Simpson: Oh, what the hell, it's me. I'm attracted to another woman. What am I going to do?
Barney_Gumble: (RECITING) "Your infatuation is based on a physical attraction. Talk to the woman, and you'll realize you have nothing in common."
Homer_Simpson: Barney. That is so insightful. How did you come up with that?
Barney_Gumble: It was on one of these bar napkins.


Homer_Simpson: Okay, we've got the secret vigilante handshake. Now we need code names. I'll be "Cueball." Skinner can be "Eightball", Barney will be "Twelveball," and Moe, you can be "Cueball."
Moe_Szyslak: You're an idiot.


Dr._Babcock: Excuse me?
Homer_Simpson: Hello, is this President Clinton? Good. I figured if anyone kn

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [6]:
##########################################
# inserted cell for lookup table doodles #
##########################################
from collections import Counter
import numpy as np

print(text[:10])
# from the text file remove the line breaks '\n' and then join back together
all_text = ' '.join(text.split('\n'))
all_text[:400] # just check this looks to have worked

# get all words that are in the text var into a list
all_words = ' '.join(text.split('\n')).split(' ')
all_words[:20] # check how these look
# count words into a dictionary
word_counts = Counter(all_words)
# check most common words
Counter(all_words).most_common(100)

# get into shape for the vocab vars
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(vocab[:20])

# a more pythonic version here
vocab_to_int = {text: i for i, text in enumerate(sorted(Counter(all_words), 
                                                        key=Counter(all_words).get, 
                                                        reverse=True), 1)} # to allow 0 padding if necesssary
int_to_vocab = {i: text for i, text in enumerate(sorted(Counter(all_words), 
                                                        key=Counter(all_words).get, 
                                                        reverse=True), 1)}

# vocab_to_int
# int_to_vocab


Moe_Szysla
['I', 'Moe_Szyslak:', 'the', 'Homer_Simpson:', 'a', 'you', 'to', '', 'of', 'and', 'my', 'in', "I'm", 'that', 'is', 'for', 'your', 'this', 'it', 'Lenny_Leonard:']


In [7]:
import numpy as np
from collections import Counter
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    # Create dictionaires
    ordered_word_dict = sorted(Counter(text), key=Counter(text).get, reverse=True)
    vocab_to_int = {text: i for i, text in enumerate(ordered_word_dict)}
    int_to_vocab = dict(enumerate(ordered_word_dict))
    
#     print(ordered_word_dict[:20])
#     print(vocab_to_int)
#     print(int_to_vocab)
    
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [8]:
########################################
# inserted cell for puntuation doodles #
########################################
token_lookup = {'.': '||period||',
                ',': '||comma||',
                '"': '||quotation_mark',
                ';': '||semi_colon||',
                '!': '||exclmation_mark',
                '?': '||question_mark||',
                '(': '||left_parentheses||',
                ')': '||right_parentheses||',
                '-': '||dash||',
                '\n':'||new_line||'}
token_lookup

{'\n': '||new_line||',
 '!': '||exclmation_mark',
 '"': '||quotation_mark',
 '(': '||left_parentheses||',
 ')': '||right_parentheses||',
 ',': '||comma||',
 '-': '||dash||',
 '.': '||period||',
 ';': '||semi_colon||',
 '?': '||question_mark||'}

In [9]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_lookup = {'.': '||period||',
                    ',': '||comma||',
                    '"': '||quotation_mark',
                    ';': '||semi_colon||',
                    '!': '||exclmation_mark',
                    '?': '||question_mark||',
                    '(': '||left_parentheses||',
                    ')': '||right_parentheses||',
                    '--':'||dash||',
                    '\n':'||new_line'}
    return token_lookup

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [13]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='Targets')
    LearningRate = tf.placeholder(tf.float32, name='LearningRate')
    
    return (Input, Targets, LearningRate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [14]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    # vars for this function
    no_lstm_layers = 2
    
    # Basic LSTM cell here
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    # Placeholder note - This is where dropout can be implemented if necessary
    # Note no use of dropout as there nothing to prevent overfitting of
    # drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # stack up number of layers here
    # Cell = tf.contrib.rnn.MultiRNNCell([drop] * no_lstm_layers)
    Cell = tf.contrib.rnn.MultiRNNCell([lstm] * no_lstm_layers)
    
    # set initial state with zeros and the batch size
    InitialState = tf.identity(Cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    return (Cell, InitialState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [15]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
    # Define the embedding matrix
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1.0, 1.0))
    
    # This is the lookup vector for each source word
    Embedded = tf.nn.embedding_lookup(embeddings, input_data)
    
    return Embedded


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [16]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    # Note inital_state already specifies in get_init_cell so just need to specify data type here
    Outputs, FinalState = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(FinalState, name='final_state')
    
    return (Outputs, FinalState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [18]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    # set embedded by calling get_embed function
    embedded = get_embed(input_data, vocab_size, embed_dim=rnn_size)
    
    # build RNN
    Outputs, FinalState = build_rnn(cell, embedded)
    
    # weights
    # using truncated_normal with mean 0 & stdev 0.1
#     weights = tf.Variable(tf.truncated_normal(shape=[rnn_size, vocab_size],
#                                              mean=0.0,
#                                              stddev=0.1,
#                                              dtype=tf.float32))
                         
                         
    # biases
#     biases = tf.Variable(tf.zeros(shape=[vocab_size], dtype=tf.float32))
    
    
    # Check shape of these objects being created so far
#     print("embedded   = {}".format(embedded))
#     print("Outputs    = {}".format(Outputs))
#     print("FinalState = {}".format(FinalState))
#     print("weights    = {}".format(weights))
#     print("biases     = {}".format(biases))
        
    # apply weights and biases to output
    # Logits = tf.add(tf.matmul(weights, Outputs), biases) 
    
    # Need to find a solution to this error
    # ValueError: Shape must be rank 2 but is rank 3 for 'MatMul' (op: 'MatMul') with input shapes: [256,27], [128,5,256]
       
    # solution to use tf.contrib.layers.fully_connected
    # initialize weights from truncated normal dist (2 stddev)
    # note on biases initialization - using 1s instead of 0s to reduce the scale of 'forgetting' in the forget gates
    # at the begininning of training
    Logits = tf.contrib.layers.fully_connected(Outputs, vocab_size,
                                               weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer=tf.ones_initializer(),
                                               activation_fn=None)
    return (Logits, FinalState)
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [19]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    no_batches = int(len(int_text) / (batch_size * seq_length))
    batch_length = batch_size * seq_length
    
    # split the in_text var into input and target vars - where target is indexed at input + 1 (target_index)
    target_index = 1
    input_data = np.array(int_text[: no_batches * batch_length])
    target_data = np.array(int_text[target_index: no_batches * batch_length + target_index])
    
    # reshape batches 
    input_batches = np.split(input_data.reshape(batch_size, -1), no_batches, 1)
    target_batches = np.split(target_data.reshape(batch_size, -1), no_batches, 1)
    
    return np.array(list(zip(input_batches, target_batches))) 


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [68]:
# Dataset Stats
# Roughly the number of unique words: 11492
# Number of scenes: 262
# Average number of sentences in each scene: 15.248091603053435
# Number of lines: 4257
# Average number of words in each line: 11.50434578341555

# Number of Epochs
num_epochs = 120
# Batch Size
batch_size = 1024 # should this simply be the largest that we can through at the model given the system memory contraints?
                  # also appears to be a factor in training length if batch sizes are smaller training takes longer
                  # smaller batch sizes allow a more regular updating of gradients at the expensive of more noise 
                  # but smaller batch sizes results in higher overall loss rate as the model converges
# RNN Size
rnn_size = 1024 # this value (vs a lower value of 512 allows a faster reduction in loss with a smalled learning rate)
# Sequence Length
seq_length = 12 # use 12 as this is the rounded integer of the average number of words in a sequence in the training data
# Learning Rate
learning_rate = 1e-2
# Show stats for every n number of batches
show_every_n_batches = 20

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [70]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [71]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/5   train_loss = 8.834
Epoch   4 Batch    0/5   train_loss = 5.047
Epoch   8 Batch    0/5   train_loss = 4.295
Epoch  12 Batch    0/5   train_loss = 3.756
Epoch  16 Batch    0/5   train_loss = 3.260
Epoch  20 Batch    0/5   train_loss = 2.478
Epoch  24 Batch    0/5   train_loss = 2.195
Epoch  28 Batch    0/5   train_loss = 1.584
Epoch  32 Batch    0/5   train_loss = 1.045
Epoch  36 Batch    0/5   train_loss = 0.786
Epoch  40 Batch    0/5   train_loss = 0.540
Epoch  44 Batch    0/5   train_loss = 0.415
Epoch  48 Batch    0/5   train_loss = 0.402
Epoch  52 Batch    0/5   train_loss = 0.346
Epoch  56 Batch    0/5   train_loss = 0.302
Epoch  60 Batch    0/5   train_loss = 0.285
Epoch  64 Batch    0/5   train_loss = 0.278
Epoch  68 Batch    0/5   train_loss = 0.274
Epoch  72 Batch    0/5   train_loss = 0.272
Epoch  76 Batch    0/5   train_loss = 0.271
Epoch  80 Batch    0/5   train_loss = 0.270
Epoch  84 Batch    0/5   train_loss = 0.268
Epoch  88 Batch    0/5   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [74]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [75]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [64]:
import random

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function

    # set a threshold for a list of max probabililities bases on the average
#     avg_prob = (sum(probabilities) / len(probabilities)) * 1.5
#     avg_prob = max(probabilities) * 1
    
#     # get words in a list
#     word_list = list(int_to_vocab.values())

#     # create a dictionary of probabilities
#     prob_dict = dict(zip(word_list, probabilities))
    
#     # take all values in the dictionary that are greater than or equal to the avg_prob
#     select_dict = {k:v for (k, v) in prob_dict.items() if v >= avg_prob}
    
#     # put these into a list
#     select_list = list(select_dict.keys())
    
#     # select a randon word from the select_list lst
#     select_word = random.choice(select_list)
    
    # get words in a list
    word_list = list(int_to_vocab.values())
    
    # using numpy.random.choice
    select_word = np.random.choice(word_list, 1, p=probabilities)[0]
    
    return select_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [80]:
#gen_length = 15 * 12 # avg no sentences in a scene, avg no words in a sentence
gen_length = 300
# homer_simpson, moe_szyslak, or barney_gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: shut up, or i'll ram a stool down your throat!
moe_szyslak:(offended) not...
homer_simpson:(grim) uh-oh...
fire_inspector: boo who gets money.
moe_szyslak: eh, what you went to the little dangerous?
barney_gumble: wow-- i can't hate him for a couple!
moe_szyslak: and i couldn't talk, this again!
seymour_skinner: i didn't want to sell the money to buy the pipes for yourself?
moe_szyslak:(sighs) well, you're a disgrace.
homer_simpson:(sing-song) you...
moe_szyslak: oh,...(indignant) i will say good day to my temple of speech.
lisa_simpson: that was chairman his body for his face and gentlemen...
homer_simpson:(increasingly breathless)...
moe_szyslak: okay, gimme that. y-you made a beer with a certificate of bills.
moe_szyslak: yeah, i can't believe some bourbon that i would...
moe_szyslak:(excited) oh, weren't ya... didn't?
moe_szyslak: no.
homer_simpson:(drunk, remorseful) i am so stupid.
homer_simpson: we'd have to be something for stock manipulation and securities who tur

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.